<a href="https://colab.research.google.com/github/tavochopadron-cyber/inteligencia-artificial/blob/main/RNN/Prediccion_de_la_Inflacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://www.banxico.org.mx/SieInternet/consultarDirectorioInternetAction.do?accion=consultarCuadro&idCuadro=CP151&sector=8&locale=es

# Importamos pandas y cargamos el archivo copiando la ruta de acceso mostrando las primeras 20 filas

In [ ]:
import pandas as pd

df = pd.read_excel('/content/Consulta_20241113-093525023.xlsx')
df.head(20)

# Seleccionamos la fila 8 del DataFrame

In [ ]:
# Seleccionar la fila 8 como encabezado (índice 7)
new_header = df.iloc[8]
new_header

# Creamos un nuevo DataFrame tomando la fila desde la 18 en adelante para eliminar datos irrelevantes

In [ ]:
# Crear un nuevo DataFrame desde la fila 17 en adelante
df = df[17:]
df

##Asignamos valores de new_header como los nombres de las columnasnmostrando las primeras 5 filas

In [ ]:
# Asignar el nuevo encabezado
df.columns = new_header
df.head()

NameError: name 'new_header' is not defined

##Reiniciamos el indice del DataFrame para que comience desde cero descartando el indice anterior e imprimiendo las primeras 5 filas

In [ ]:
# Reiniciar el índice para que comience desde 0 en el nuevo DataFrame
df.reset_index(drop=True, inplace=True)
df.head()

NameError: name 'df' is not defined

Convertimos la columna "Titulo" a tipo datetime y la guardamos en una nueva columna llamada "Fecha"

In [ ]:
# Convertir la columna 'fecha' a tipo datetime
df['Fecha'] = pd.to_datetime(df['Título'])
df.head()

NameError: name 'df' is not defined

Mostramos el resumen del DataFrame para ver la estructura y detectar que nos falta

In [ ]:
df.info()

##Filtramos el DataFrame para quedarnos con la fila "Fecha" del el 1 de enero de 2014 y el 31 de diciembre de 2024

In [ ]:
 # Filtrar por el año 2023
_df = df[(df['Fecha'] >= '01-01-2014') & (df['Fecha'] <= '12-31-2024')]
_df.head()

NameError: name 'df' is not defined

# Mostramos las primeras 5 filas para ver si se hizo el filtrado

In [ ]:
_df.tail()

##Creamos una grafica colocando las etiquetas para mostrar como vario el indice a lo largo de los años

In [ ]:
import matplotlib.pyplot as plt

# dates = pd.to_datetime(_df["Fecha"]).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()
dates = _df["Fecha"].tolist()

# Grafica el "Índice Nacional de Precios al consumidor, variación anual"
plt.figure(figsize=(16, 10))
plt.plot(dates,_df["Índice Nacional de Precios al consumidor, variación anual"], color='blue', label="Variación anual")
plt.title("Índice Nacional de Precios al Consumidor - variación anual")
plt.xlabel("Tiempo")
plt.ylabel("Variación anual")
plt.xticks(rotation=90)
plt.legend()
plt.show()

# Separamos los valores de entrada y variable de objet

In [ ]:
X = _df.drop(columns=["Índice Nacional de Precios al consumidor, variación anual", "Título", "Fecha"])
y = _df["Índice Nacional de Precios al consumidor, variación anual"].values

NameError: name '_df' is not defined

# Mostramos cuantas filas y columnas contiene el DataFrame

In [ ]:
X.shape

NameError: name 'X' is not defined

# Normalizamos los datos para que los valores queden entre 0 y 1

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

# Construimos secuencias, ponemos 12 secuencias alo largo del tiempo

In [ ]:
import numpy as np

window_size = 12
X_rnn = []
y_rnn = []
# Construye secuencias temporales
for i in range(window_size, len(X_scaled)):
    X_rnn.append(X_scaled[i - window_size:i])
    y_rnn.append(y_scaled[i])

X_rnn, y_rnn = np.array(X_rnn), np.array(y_rnn)

# Dividimos los datos para entrenamiento y prueba

In [ ]:
split = int(len(X) * 0.8)
X_train, y_train = X_rnn[:split], y_rnn[:split]
X_test, y_test = X_rnn[split:], y_rnn[split:]

NameError: name 'X' is not defined

# Mostramos las dimensiones del modelo x

In [ ]:
X_train.shape

# Mostramos las dimensiones del arreglo mostrando el numero de secuencias para y

In [ ]:
y_test.shape

# Indicamos que cada 12 se hace una secucncia temporal en 12 pasos de tiempo

In [ ]:
window_size

NameError: name 'window_size' is not defined

Mostramos el numero de caracteristicas por paso del tiempo para decir cuantas columnas tiene cada registro

In [ ]:
X_train.shape[2]

# Construimos la red neuronal: la primera capa con 60 neuronas, la segunda con 30, la tercera con 15

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, GRU

model = Sequential([
  SimpleRNN(units=60, return_sequences=True, input_shape=(window_size, X_train.shape[2])),
  SimpleRNN(units=30, return_sequences=True),
  SimpleRNN(units=15, return_sequences=False),
  Dense(units=1)
])


# Optimizamos el modelo con Adam con una tasa de aprendizaje de 0.001

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compilar el modelo
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

# Entrenamos el modelo durante 10 epocas con tamaño de 1 batch

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, batch_size=1, epochs=10, validation_data=(X_test, y_test))

# Realizamos las predicciones, descalonamos los valores reales para devolver los valores originales imprimiendo los primeros 5 valores

In [ ]:
# Predice en el conjunto de prueba y desescala los datos
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_original = scaler.inverse_transform(y_test)

# Muestra algunas predicciones y valores reales
print("Predicciones:", predictions.flatten()[:5])
print("Valores reales:", y_test_original.flatten()[:5])


# Evaluamos el desempeño del modelo calculando el error absoluto medio, error cuadratico, raiz del error cuadratico medio y el coeficiente de determinacion

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Calcula el MAE
mae = mean_absolute_error(y_test, predictions)
print(f"Error Absoluto Medio (MAE): {mae}")

# Calcula el MSE
mse = mean_squared_error(y_test, predictions)
print(f"Error Cuadrático Medio (MSE): {mse}")

# Calcula el RMSE
rmse = np.sqrt(mse)
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse}")

# Calcula el Coeficiente de Determinación (R²)
r2 = r2_score(y_test, predictions)
print(f"Coeficiente de Determinación (R²): {r2}")

# Creamos una garfica para comparar los valores reales de la inflacion real y predicha a lo largo del tiempo mostrando el desempeño del modelo en el conjunto de prueba

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Gráfica de los valores reales y las predicciones del conjunto de prueba
valid = _df[split:]
valid = valid.reset_index(drop=True)
valid['Predictions'] = np.nan
valid.loc[window_size:, 'Predictions'] = predictions

dates_valid = pd.to_datetime(valid['Fecha']).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

plt.figure(figsize=(16, 8))
plt.title('Modelo RNN para Predicción de la inflación')
plt.xlabel('Fecha')
plt.ylabel('Inflacion')
plt.plot(dates_valid, valid[["Índice Nacional de Precios al consumidor, variación anual", 'Predictions']])
plt.legend(['Valor Real', 'Predicciones'], loc='lower right')
plt.xticks(rotation=90)
plt.show()